In [1]:
!pip install transformers[torch]
!pip install datasets
!pip install sentence_transformers
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 29.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.6 MB/s eta 0:00:00

In [2]:
import pandas as pd
import numpy as np

In [3]:
from numba import cuda
import pandas as pd
import numpy as np
from transformers import DataCollatorForSeq2Seq
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import create_optimizer
from tqdm import tqdm
import gc
from transformers import get_scheduler
from torch.optim import AdamW
import torch.cuda
from google.colab import runtime


# Preprocessing

In [4]:
dc = pd.read_csv('/content/DecOp_data_EN_500.csv', sep=';', encoding = "ISO-8859-1")
dc.head()

d1 = []
for i in range(dc.shape[0]):
  row = dc.iloc[i]
  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['A'].replace('\n', " ") ,
  'labels'  : row['GT.A']})

  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['E'].replace('\n', " ")  ,
  'labels'  : row['GT.E']})
  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['GM'].replace('\n', " ")  ,
  'labels'  : row['GT.GM']})

  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['Pom'].replace('\n', " ")  ,
  'labels'  : row['GT.Pom']})

  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['CL'].replace('\n', " ")  ,
  'labels'  : row['GT.CL']})

decop = pd.DataFrame.from_records(d1)

decop=decop[['ID', 'sent', 'labels']]
decop['type']='A'

In [5]:
hc = pd.read_csv('/content/preprocessed_hcV3.csv', sep=';', encoding = "ISO-8859-1")

mem = hc[hc['memType']!='retold']
mem = mem.dropna(subset=['story', 'memType'])
mem = mem[['story', 'memType']]
mem['memType'][mem['memType']=='recalled']='T'
mem['memType'][mem['memType']=='imagined']='F'
mem = mem.rename(columns={'story': 'sent', 'memType': 'labels'})
mem['type']='B'

In [6]:
intent = pd.read_csv('/content/sign_events_data_statements.csv', encoding = "ISO-8859-1")


intent['outcome_class'][intent['outcome_class']=='t']='T'
intent['outcome_class'][intent['outcome_class']=='d']='F'
intent['q1']=intent['q1'].apply(lambda x: x.replace('\n', ''))
intent = intent.rename(columns={'q1': 'sent', 'outcome_class': 'labels'})
intent = intent[['sent', 'labels']]
intent['type']='C'

<ipython-input-6-ba4812cfd6fa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intent['outcome_class'][intent['outcome_class']=='t']='T'
<ipython-input-6-ba4812cfd6fa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intent['outcome_class'][intent['outcome_class']=='d']='F'


In [7]:
def create_dataset(Scenario = {'train':[], 'test':[], 'scenario':1}, decop=None, mem = None, intent = None, seed=42, cv=10):

  train={f'split_{i+1}':[] for i in range(cv)}
  test={f'split_{i+1}':[] for i in range(cv)}

  np.random.seed(seed)

  de_arr = np.arange(1, 501)
  np.random.shuffle(de_arr)
  de_arr_r = de_arr.reshape((cv, int(500/cv)))

  num_row = (mem.index.shape[0] - (mem.shape[0]%cv))
  me_arr = np.random.choice(mem.index.shape[0], int(num_row), replace=False)
  me_arr_r = me_arr.reshape((cv, int(num_row/cv)))

  in_arr = np.arange(0, intent.index.shape[0])
  np.random.shuffle(in_arr)
  in_arr_r = in_arr.reshape((cv, int(intent.shape[0]/cv))) ### fin qui testata

  cv_ind = 0
  for k in train.keys():



    if Scenario['scenario'] == 1 or Scenario['scenario'] == 3:

      if 'decop' in Scenario['train']:
        ind = de_arr_r[cv_ind,:]
        antind = [i for i in de_arr if i not in ind]

        decop_train = decop[decop['ID'].isin(antind)]
        decop_test = decop[decop['ID'].isin(ind)]

        train[k].append(decop_train)
        test[k].append(decop_test)

      if 'hyppocorpus' in Scenario['train']:
        ind = me_arr_r[cv_ind,:]
        antind = [i for i in me_arr if i not in ind]

        mem_train = mem[mem.index.isin(antind)]
        mem_test = mem[mem.index.isin(ind)]

        train[k].append(mem_train)
        test[k].append(mem_test)

      if 'intention' in Scenario['train']:

        ind = in_arr_r[cv_ind,:]
        antind = [i for i in in_arr if i not in ind]

        intent_train = intent[intent.index.isin(antind)]
        intent_test = intent[intent.index.isin(ind)]

        train[k].append(intent_train)
        test[k].append(intent_test)

      train[k] = pd.concat(train[k])
      test[k] = pd.concat(test[k])

    else:

      if 'decop' in Scenario['train']:
        train[k].append(decop)
      elif 'decop' in Scenario['test']:
        test[k].append(decop)

      if 'hyppocorpus' in Scenario['train']:
        train[k].append(mem)
      elif 'hyppocorpus' in Scenario['test']:
        test[k].append(mem)

      if 'intention' in Scenario['train']:
        train[k].append(intent)
      elif 'intention' in Scenario['test']:
        test[k].append(intent)

      train[k] = pd.concat(train[k])
      test[k] = test[k][0]

    cv_ind+=1



  return train, test



In [8]:
def preprocess_function(examples):
    inputs = [ex for ex in examples["sent"]]
    targets = [ex for ex in examples["labels"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, truncation=True)
    return model_inputs

# Experiment Scenario 1 Decop
## Base model

In [12]:
seed=42
np.random.seed(seed)
collect_result={}
numcv=1
num_epochs = 3
model_size='base'

Scenarios = {1: {'train': ['decop', 'hyppocorpus'], 'test': ['intention'],  'scenario': 2},
             2: {'train': ['decop', 'intention'], 'test': ['hyppocorpus'],  'scenario': 2},
             3: {'train': ['hyppocorpus', 'intention'], 'test': ['decop'],  'scenario': 2}}
results={}
types={}
preds={}
trus={}

for Scenario in Scenarios.values():
  train, test = create_dataset(Scenario, decop=decop, mem=mem, intent=intent, seed=seed, cv=numcv)
  for sp in tqdm(train.keys()):


    data_train = Dataset.from_pandas(train[sp])
    data_test = Dataset.from_pandas(test[sp])

    checkpoint = f"google/flan-t5-{model_size}"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

    data_train = data_train.map(preprocess_function, batched=True)
    data_test = data_test.map(preprocess_function, batched=True)

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)



    training_args = Seq2SeqTrainingArguments(
        output_dir=f"{model_size}_scenario_{Scenario['scenario']}",
        evaluation_strategy='epoch',
        learning_rate=5e-5,
        per_device_train_batch_size=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        fp16=False,
        push_to_hub=False,
    )
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=data_train,
        eval_dataset = data_test,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train()

    pred = []
    for i  in test[sp]['sent']:
      toki = tokenizer(i,  return_tensors="pt").input_ids
      h = model.generate(toki.cuda(), return_dict_in_generate=True, output_scores=True)
      decoded_preds = tokenizer.batch_decode(h.sequences, skip_special_tokens=True)
      pred.append(decoded_preds[0])

    pred = np.array(pred)


    types[sp] = data_test['type']
    preds[sp] = pred
    trus[sp] = np.array(test[sp]['labels'])

    collect_result[sp]=pd.DataFrame.from_dict({"prediction": pred, 'labels': np.array(test[sp]['labels']), 'type': test[sp]['type']})
    collect_result[sp]['corr'] = collect_result[sp]['prediction'] == collect_result[sp]['labels']
    results[sp] = [(pred == trus[sp]).mean()]
    del model
    del trainer
    torch.cuda.empty_cache()

  print('#################################')
  print(f"Train: {Scenario['train']}, Test: {Scenario['test']}")
  results = pd.DataFrame.from_dict(results)
  print('Average accuracy: ', results.mean(axis=1))
  print('Standard deviation of accuracy: ', results.std(axis=1))
  print('Max accuracy: ', results.max(axis=1))
  print('Min deviation of accuracy: ', results.min(axis=1))

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/8006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1640 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.465700,0.684767
2,0.323900,1.059258
3,0.278400,1.291819


...

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.442400,0.720669
2,0.360300,0.901434
3,0.321700,1.188255


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1/1 [29:35<00:00, 1775.41s/it]

#################################
Train: ['hyppocorpus', 'intention'], Test: ['decop']
Average accuracy:  0    0.494
dtype: float64
Standard deviation of accuracy:  0   NaN
dtype: float64
Max accuracy:  0    0.494
dtype: float64
Min deviation of accuracy:  0    0.494
dtype: float64


# Experiment Scenario 1 Decop
## Small model

In [13]:
seed=42
np.random.seed(seed)
collect_result={}
numcv=1
num_epochs = 3
model_size='small'

Scenarios = {1: {'train': ['decop', 'hyppocorpus'], 'test': ['intention'],  'scenario': 2},
             2: {'train': ['decop', 'intention'], 'test': ['hyppocorpus'],  'scenario': 2},
             3: {'train': ['hyppocorpus', 'intention'], 'test': ['decop'],  'scenario': 2}}
results={}
types={}
preds={}
trus={}

for Scenario in Scenarios.values():
  train, test = create_dataset(Scenario, decop=decop, mem=mem, intent=intent, seed=seed, cv=numcv)
  for sp in tqdm(train.keys()):


    data_train = Dataset.from_pandas(train[sp])
    data_test = Dataset.from_pandas(test[sp])

    checkpoint = f"google/flan-t5-{model_size}"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

    data_train = data_train.map(preprocess_function, batched=True)
    data_test = data_test.map(preprocess_function, batched=True)

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)



    training_args = Seq2SeqTrainingArguments(
        output_dir=f"{model_size}_scenario_{Scenario['scenario']}",
        evaluation_strategy='epoch',
        learning_rate=5e-4,
        per_device_train_batch_size=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        fp16=False,
        push_to_hub=False,
    )
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=data_train,
        eval_dataset = data_test,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train()

    pred = []
    for i  in test[sp]['sent']:
      toki = tokenizer(i,  return_tensors="pt").input_ids
      h = model.generate(toki.cuda(), return_dict_in_generate=True, output_scores=True)
      decoded_preds = tokenizer.batch_decode(h.sequences, skip_special_tokens=True)
      pred.append(decoded_preds[0])

    pred = np.array(pred)


    types[sp] = data_test['type']
    preds[sp] = pred
    trus[sp] = np.array(test[sp]['labels'])

    collect_result[sp]=pd.DataFrame.from_dict({"prediction": pred, 'labels': np.array(test[sp]['labels']), 'type': test[sp]['type']})
    collect_result[sp]['corr'] = collect_result[sp]['prediction'] == collect_result[sp]['labels']
    results[sp] = [(pred == trus[sp]).mean()]
    del model
    del trainer
    torch.cuda.empty_cache()


  results = pd.DataFrame.from_dict(results)
  print('#################################')
  print(f"Train: {Scenario['train']}, Test: {Scenario['test']}")
  print('Average accuracy: ', results.mean(axis=1))
  print('Standard deviation of accuracy: ', results.std(axis=1))
  print('Max accuracy: ', results.max(axis=1))
  print('Min deviation of accuracy: ', results.min(axis=1))

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/8006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1640 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.355200,0.532155
2,0.320000,0.573468
3,0.227800,0.774565


...

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.344700,0.410992
2,0.282600,0.627767
3,0.251900,0.877111


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1/1 [18:09<00:00, 1089.91s/it]

#################################
Train: ['hyppocorpus', 'intention'], Test: ['decop']
Average accuracy:  0    0.5312
dtype: float64
Standard deviation of accuracy:  0   NaN
dtype: float64
Max accuracy:  0    0.5312
dtype: float64
Min deviation of accuracy:  0    0.5312
dtype: float64
